In [2]:
import numpy as np
from osgeo import gdal
import os

def calculate_raster_change_detection(raster_2023_path, raster_2024_path, raster_2025_path, output_dir):
    """
    Calculates the difference between three time-series rasters and exports the
    change maps to the specified output directory.

    Args:
        raster_2023_path (str): File path for the 2023 raster (T0).
        raster_2024_path (str): File path for the 2024 raster (T1).
        raster_2025_path (str): File path for the 2025 raster (T2).
        output_dir (str): Directory where the difference rasters will be saved.
    """

    print("--- Starting Change Detection Analysis ---")

    # --- 1. Load Rasters and Read Data ---
    
    # Open all three rasters
    ds_2023 = gdal.Open(raster_2023_path, gdal.GA_ReadOnly)
    ds_2024 = gdal.Open(raster_2024_path, gdal.GA_ReadOnly)
    ds_2025 = gdal.Open(raster_2025_path, gdal.GA_ReadOnly)

    if not all([ds_2023, ds_2024, ds_2025]):
        print("Error: Could not open one or more input rasters. Check file paths.")
        return

    # Extract metadata from 2023 (assuming all are identical)
    geo_transform = ds_2023.GetGeoTransform()
    projection = ds_2023.GetProjection()
    x_size = ds_2023.RasterXSize
    y_size = ds_2023.RasterYSize
    # data_type is only used for reference, output is GDT_Float32
    data_type = ds_2023.GetRasterBand(1).DataType 
    nodata_value_in = ds_2023.GetRasterBand(1).GetNoDataValue()
    
    # Read data into NumPy arrays (force float32 for subtraction results)
    R2023 = ds_2023.GetRasterBand(1).ReadAsArray().astype(np.float32)
    R2024 = ds_2024.GetRasterBand(1).ReadAsArray().astype(np.float32)
    R2025 = ds_2025.GetRasterBand(1).ReadAsArray().astype(np.float32)

    # Convert nodata_value to standard Python float (double) to satisfy GDAL's SetNoDataValue API requirement
    if nodata_value_in is None:
        # If NoData is not defined, use a safe default and cast it
        nodata_value = -9999.0 
    else:
        nodata_value = float(nodata_value_in) # Ensure standard Python float (double)

    # --- 2. Create Common NoData Mask ---
    
    # Identify valid data (non-NoData) across ALL three rasters
    valid_mask = (R2023 != nodata_value) & \
                 (R2024 != nodata_value) & \
                 (R2025 != nodata_value)
    
    # --- 3. Calculate Differences ---
    
    # Initialize difference arrays with the NoData value
    Diff_23_24 = np.full_like(R2023, nodata_value, dtype=np.float32)
    Diff_24_25 = np.full_like(R2023, nodata_value, dtype=np.float32)
    Diff_Cumulative = np.full_like(R2023, nodata_value, dtype=np.float32)

    print("Calculating R2024 - R2023...")
    # Calculate difference only where data is valid in both 2023 and 2024
    Diff_23_24[valid_mask] = R2024[valid_mask] - R2023[valid_mask]

    print("Calculating R2025 - R2024...")
    # Calculate difference only where data is valid in both 2024 and 2025
    Diff_24_25[valid_mask] = R2025[valid_mask] - R2024[valid_mask]
    
    print("Calculating Cumulative Change R2025 - R2023...")
    # Calculate total cumulative difference
    Diff_Cumulative[valid_mask] = R2025[valid_mask] - R2023[valid_mask]

    # --- 4. Define Output Function ---

    def write_raster(array, output_filename, nodata):
        """Helper function to write a NumPy array to GeoTIFF."""
        output_path = os.path.join(output_dir, output_filename)
        
        driver = gdal.GetDriverByName("GTiff")
        out_ds = driver.Create(
            output_path, 
            x_size, 
            y_size, 
            1, 
            gdal.GDT_Float32, # Use Float32 for difference rasters to handle negative values and precision
            options = ['COMPRESS=LZW', 'TILED=YES']
        )
        
        out_ds.SetProjection(projection)
        out_ds.SetGeoTransform(geo_transform)

        out_band = out_ds.GetRasterBand(1)
        out_band.WriteArray(array)
        # Fix for TypeError: nodata is now guaranteed to be a standard Python float
        out_band.SetNoDataValue(nodata)
        out_band.FlushCache()
        
        # Clean up GDAL reference
        del out_ds 
        print(f"✅ Saved: {output_filename}")


    # --- 5. Export Results ---
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
        
    write_raster(Diff_23_24, "change_2024_minus_2023.tif", nodata_value)
    write_raster(Diff_24_25, "change_2025_minus_2024.tif", nodata_value)
    write_raster(Diff_Cumulative, "cumulative_change_2025_minus_2023.tif", nodata_value)

    # Clean up input GDAL references
    del ds_2023, ds_2024, ds_2025
    
    print("--- Change Detection Complete ---")

# --- EXAMPLE USAGE ---
# NOTE: Replace these placeholder paths with your actual file locations and desired output folder.
RASTER_2023 = "E:/GOES-R Lightning Data/JOINED-MEAN-FINAL/mean_energy_2023_wgs84.tif"
RASTER_2024 = "E:/GOES-R Lightning Data/JOINED-MEAN-FINAL/mean_energy_2024_wgs84.tif"
RASTER_2025 = "E:/GOES-R Lightning Data/JOINED-MEAN-FINAL/mean_energy_2025_wgs84.tif"
OUTPUT_FOLDER = "E:/GOES-R Lightning Data/JOINED-MEAN-FINAL/"

# Uncomment the line below with your paths to run the function:
calculate_raster_change_detection(RASTER_2023, RASTER_2024, RASTER_2025, OUTPUT_FOLDER)


--- Starting Change Detection Analysis ---
Calculating R2024 - R2023...
Calculating R2025 - R2024...
Calculating Cumulative Change R2025 - R2023...
✅ Saved: change_2024_minus_2023.tif
✅ Saved: change_2025_minus_2024.tif
✅ Saved: cumulative_change_2025_minus_2023.tif
--- Change Detection Complete ---
